In [1]:
from keras.preprocessing.image import ImageDataGenerator

Using Theano backend.
ERROR (theano.gpuarray): Could not initialize pygpu, support disabled
Traceback (most recent call last):
  File "/Users/paragtaneja/anaconda2/lib/python2.7/site-packages/theano/gpuarray/__init__.py", line 164, in <module>
    use(config.device)
  File "/Users/paragtaneja/anaconda2/lib/python2.7/site-packages/theano/gpuarray/__init__.py", line 151, in use
    init_dev(device)
  File "/Users/paragtaneja/anaconda2/lib/python2.7/site-packages/theano/gpuarray/__init__.py", line 60, in init_dev
    sched=config.gpuarray.sched)
  File "pygpu/gpuarray.pyx", line 634, in pygpu.gpuarray.init
  File "pygpu/gpuarray.pyx", line 584, in pygpu.gpuarray.pygpu_init
  File "pygpu/gpuarray.pyx", line 1057, in pygpu.gpuarray.GpuContext.__cinit__
GpuArrayException: cuInit: (null): (null)


In [2]:
img_width, img_height = 250, 250

train_data_dir = '/Users/paragtaneja/AnacondaProjects/Sketch/Train1'
validation_data_dir = '/Users/paragtaneja/AnacondaProjects/Sketch/Test1'
print(validation_data_dir)

/Users/paragtaneja/AnacondaProjects/Sketch/Test1


In [3]:
# used to rescale the pixel values from [0, 255] to [0, 1] interval
from keras import backend as K
K.set_image_dim_ordering('th')
datagen = ImageDataGenerator(rescale=1./255)

# automagically retrieve images and their classes for train and validation sets
train_generator = datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_width, img_height),
        batch_size=32,
        class_mode='binary')

validation_generator = datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_width, img_height),
        batch_size=32,
        class_mode='binary')

Found 240 images belonging to 3 classes.
Found 96 images belonging to 3 classes.


In [8]:
from keras.models import Sequential,Model
from keras.layers import Input, Dense, Conv2D, MaxPooling2D, AveragePooling2D,UpSampling2D, ZeroPadding2D, Convolution2D, Deconvolution2D, merge
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.models import load_model
from keras import objectives, layers



In [ ]:
model = applications.VGG16(weights='imagenet', include_top=False)
print('Model loaded.')

# build a classifier model to put on top of the convolutional model
top_model = Sequential()
top_model.add(Flatten(input_shape=model.output_shape[1:]))
top_model.add(Dense(256, activation='relu'))
top_model.add(Dropout(0.5))
top_model.add(Dense(1, activation='sigmoid'))

# note that it is necessary to start with a fully-trained
# classifier, including the top classifier,
# in order to successfully do fine-tuning
top_model.load_weights(top_model_weights_path)

# add the model on top of the convolutional base
model.add(top_model)

In [ ]:
top_model = Sequential()
top_model.add(Flatten(input_shape=model.output_shape[1:]))
top_model.add(Dense(256, activation='relu'))
top_model.add(Dropout(0.5))
top_model.add(Dense(1, activation='sigmoid'))

# note that it is necessary to start with a fully-trained
# classifier, including the top classifier,
# in order to successfully do fine-tuning
top_model.load_weights(top_model_weights_path)

# add the model on top of the convolutional base
model.add(top_model)

In [5]:
#from keras import backend as K
#K.set_image_dim_ordering('th')

input = Input(shape=(3, 250, 250))

layer1_conv_1 = Convolution2D(64, 15, 15,subsample=(3, 3),activation='relu')(input)
layer1_pool_1 = MaxPooling2D(pool_size=(3, 3),strides=(2, 2))(layer1_conv_1)

layer2_conv_1 = Convolution2D(128, 5, 5,subsample=(1, 1),activation='relu')(layer1_pool_1)
layer2_pool_1 = MaxPooling2D(pool_size=(3, 3),strides=(2, 2))(layer2_conv_1)

layer3_conv_1 = Convolution2D(256, 5, 5,subsample=(1, 1),activation='relu')(layer2_pool_1)
layer3_pool_1 = MaxPooling2D(pool_size=(3, 3),strides=(2, 2))(layer3_conv_1)

#tower A
sparse_conv_a1 = Convolution2D(48, 1, 1)(layer3_pool_1)
sparse_conv_a2 = Convolution2D(64, 3, 3)(sparse_conv_a1)


# Tower B
sparse_pool_b1 = AveragePooling2D(pool_size=(3, 3), strides=(1, 1))(layer3_pool_1)
sparse_conv_b2 = Convolution2D(64, 1, 1)(sparse_pool_b1)

# Tower C
sparse_conv_c1 = Convolution2D(64, 3, 3)(layer3_pool_1)

merge_layer = merge([sparse_conv_a2, sparse_conv_b2, sparse_conv_c1], mode='concat', concat_axis=1)

layer5_pool_1 = MaxPooling2D(pool_size=(3, 3),strides=(2, 2))(merge_layer)
flat = Flatten()(layer5_pool_1)
fc1 = Dense(256, activation='relu')(flat)
dr = Dropout(0.5)(fc1)
fc2 = Dense(250, activation='sigmoid')(dr)
model = Model(input,fc2)



/Users/paragtaneja/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:6: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (15, 15), strides=(3, 3), activation="relu")`
  
/Users/paragtaneja/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:9: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, (5, 5), strides=(1, 1), activation="relu")`
  if __name__ == '__main__':
/Users/paragtaneja/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:12: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(256, (5, 5), strides=(1, 1), activation="relu")`
  if sys.path[0] == '':
/Users/paragtaneja/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:16: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(48, (1, 1))`
  app.launch_new_instance()
/Users/paragtaneja/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:17: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3,

In [6]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 3, 250, 250)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 64, 79, 79)   43264       input_1[0][0]                    
__________________________________________________________________________________________________
max_pooling2d_1 (MaxPooling2D)  (None, 64, 39, 39)   0           conv2d_1[0][0]                   
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 128, 35, 35)  204928      max_pooling2d_1[0][0]            
__________________________________________________________________________________________________
max_poolin

In [7]:
from keras.callbacks import ModelCheckpoint
#from keras import backend as K
#K.set_image_dim_ordering('th')

filepath="weights-improvement-{epoch:02d}-{val_acc:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True,  mode='max', period=1)
callbacks_list = [checkpoint]
#init_from = '/Users/paragtaneja/AnacondaProjects/Untitled Folder/first_try.h5',
model.fit_generator(
        train_generator,
        steps_per_epoch=2000//32,
        epochs=100,
        validation_data=validation_generator,
        validation_steps=800//32,
        callbacks=callbacks_list,
        verbose=0)
model.save_weights('first_try.h5')

ValueError: Error when checking target: expected dense_2 to have shape (None, 250) but got array with shape (32, 1)

In [13]:
from keras import backend as K
K.set_image_dim_ordering('th')

model = Sequential()
model.add(Conv2D(32, (3, 3),input_shape=(3,150, 150)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

In [6]:
model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

In [18]:
from keras import backend as K
K.set_image_dim_ordering('th')

model = Sequential()
model.add(Conv2D(32, (3, 3),input_shape=(3,150, 150),padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

# Conv2 16 16 (64)
model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

# Conv2 8 8 (128)
model.add(Conv2D(128, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(128, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))



In [25]:
def generator_model(input_img):

    # Encoder
    x = Conv2D(32, (3, 3), activation='relu', padding='same')(input_img)
    x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
    x = MaxPooling2D((3, 3), padding='same')(x)

    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block2_conv1')(x)
    x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    x = MaxPooling2D((3, 3), padding='same')(x)

    x = Conv2D(128, (3, 3), activation='relu', padding='same', name='block3_conv1')(x)
    x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
    x = MaxPooling2D((3, 3), padding='same')(x)

    x = Conv2D(256, (3, 3), activation='relu', padding='same', name='block4_conv1')(x)
    res = Conv2D(256, (3, 3), activation='relu', padding='same')(x)
    x = layers.add([x, res])
    res = Conv2D(256, (3, 3), activation='relu', padding='same')(x)
    encoded = layers.add([x, res])

    # Decoder
    res = Conv2D(256, (3, 3), activation='relu', padding='same', name='block5_conv1')(encoded)
    x = layers.add([encoded, res])
    res = Conv2D(256, (3, 3), activation='relu', padding='same')(x)
    x = layers.add([x, res])
    res = Conv2D(256, (3, 3), activation='relu', padding='same')(x)
    x = layers.add([x, res])

    x = Conv2D(128, (3, 3), activation='relu', padding='same', name='block6_conv1')(x)
    x = UpSampling2D((3, 3))(x)

    x = Conv2D(128, (3, 3), activation='relu', padding='same', name='block7_conv1')(x)
    res = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
    x = layers.add([x, res])
    res = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
    x = layers.add([x, res])

    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block8_conv1')(x)
    x = UpSampling2D((3, 3))(x)

    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='block9_conv1')(x)
    res = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    x = layers.add([x, res])
    res = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    x = layers.add([x, res])

    x = Conv2D(32, (3, 3), activation='relu', padding='same', name='block10_conv1')(x)
    x = UpSampling2D((3, 3))(x)

    x = Conv2D(32, (3, 3), activation='relu', padding='same', name='block11_conv1')(x)
    res = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
    x = layers.add([x, res])
    decoded = Conv2D(32, (3, 3), activation='sigmoid', padding='same')(x)
    model = Model(input=input_img, output=decoded)
    return model

In [29]:
def discriminator_model(img_input):
    
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='d_block1_conv1')(img_input)
    x = Conv2D(64, (3, 3), activation='relu', padding='same', name='d_block1_conv2')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='d_block1_pool')(x)

    # Block 2
    x = Conv2D(128, (3, 3), activation='relu', padding='same', name='d_block2_conv1')(x)
    x = Conv2D(128, (3, 3), activation='relu', padding='same', name='d_block2_conv2')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='d_block2_pool')(x)

    # Block 3
    x = Conv2D(256, (3, 3), activation='relu', padding='same', name='d_block3_conv1')(x)
    x = Conv2D(256, (3, 3), activation='relu', padding='same', name='d_block3_conv2')(x)
    x = Conv2D(256, (3, 3), activation='relu', padding='same', name='d_block3_conv3')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='d_block3_pool')(x)

    # Block 4
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='d_block4_conv1')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='d_block4_conv2')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='d_block4_conv3')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='d_block4_pool')(x)

    # Block 5
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='d_block5_conv1')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='d_block5_conv2')(x)
    x = Conv2D(512, (3, 3), activation='relu', padding='same', name='d_block5_conv3')(x)
    x = MaxPooling2D((2, 2), strides=(2, 2), name='d_block5_pool')(x)
    
    x = Flatten(name='flatten')(x)
    x = Dense(512, activation='relu', name='d_fc1')(x)
    x = Dropout(0.5)(x)
    x = Dense(1, activation='relu', name='d_fc2')(x)
    model = Model(input=img_input, output=x)
    
    return model

In [30]:
# FC

input_img = Input(shape=(3, 300, 300))
#generator = generator_model(input_img)
discrminator = discriminator_model(input_img)

model = Sequential()
#model.add(generator)
model.add(discrminator)

#model.add(Flatten())
#model.add(Dense(128))
#model.add(Activation('relu'))
#model.add(Dropout(0.5))
#model.add(Dense(64))
#model.add(Activation('relu'))
#model.add(Dropout(0.5))
#model.add(Dense(1))
#model.add(Activation('softmax'))

/Users/paragtaneja/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:34: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=Elemwise{m..., inputs=/input_7)`
